<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Workshop: Microsoft SQL Server Machine Learning Services

#### <i>A Microsoft Course from the SQL Server team</i>

## SQL Server Machine Learning Services Architecture

<p style="border-bottom: 1px solid lightgrey;"></p>


<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true">Understanding the SQL Server ML Services Architecture</h2>

SQL Server 2016 introduced the R language as a companion server alongside the SQL Server Instance - called a *satellite*. SQL Server 2017 introduced Python language support, in the same fashion and usage as R. In SQL Server 2019, Java was added as an additional satellite process.


<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true">The SQL Server Exensibility Framework</h2>

If selected, the Installer program for SQL Server installs the Microsoft ML Server-supported language runtime environments *alongside* the SQL Server Instance, and then sets up a Service (called *SQL Launchpad*) allowing the sandboxed processes to communicate over a secure channel. It also sets up several other components to allow the *scoring*, and in some cases the *training*, of Machine Learning models using Python or R. 

<img src="https://docs.microsoft.com/en-us/sql/advanced-analytics/media/generic-architecture.png?view=sql-server-2017">

When the R or Python language is called via a special Stored Procedure (which you must enable) SQL Server transfers data to the R or Python process which runs the code, and returns the result to the Stored Procedure in SQL Server.

<p>
<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/TSQLAndR.png?raw=true" width="500">
<p>

You can run code to execute in Python as well as R. You can use either language by simply setting a parameter in the Stored Procedure.

This allows SQL Server professionals to work with and hybrid data in the way they are familiar with, and the Data Scientist to develop their R or Python code anywhere, and then deploying that code to SQL Server by embedding it in a Stored Procedure.

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Enable external script execution in SQL Server</b></p>

The following code enables the external processes shown above to run on a SQL Server Instance:

In [0]:
/* Enable ML Services
NOTE: 
You must have SQL Server ML Services Installed,
The SQL server Launchpad Service must be running, 
and you may need to restart the SQL Server Service if the scripts below do not work. 
More detailed information here: https://docs.microsoft.com/en-us/sql/advanced-analytics/install/sql-machine-learning-services-windows-install?view=sql-server-ver15 
*/

EXEC sp_configure  'external scripts enabled', 1
RECONFIGURE WITH OVERRIDE

In [0]:
/* Test R */
EXEC sp_execute_external_script  @language =N'R',
@script=N'
OutputDataSet <- InputDataSet;
',
@input_data_1 =N'SELECT 1 AS [Is R Working]'
WITH RESULT SETS (([Is R Working] int not null));
GO

In [0]:
/* Test Python */
EXEC sp_execute_external_script  @language =N'Python',
@script=N'
OutputDataSet = InputDataSet;
',
@input_data_1 =N'SELECT 1 AS [Is Python Working]'
WITH RESULT SETS (([Is Python Working] int not null));
GO

In [0]:
/* Get R Info */
EXECUTE sp_execute_external_script @language = N'R'
, @script = N'
OutputDataSet <- data.frame(installed.packages()[,c("Package", "Version", "Depends", "License", "LibPath")]);'
WITH RESULT SETS(
    (Package NVARCHAR(255)
    , Version NVARCHAR(100)
    , Depends NVARCHAR(4000)
    , License NVARCHAR(1000)
    , LibPath NVARCHAR(2000))
    );
GO

In [0]:
/* Get Python Info */
EXECUTE sp_execute_external_script
@language =N'Python',
@script=N'import sys
print(sys.version)';
GO

EXECUTE sp_execute_external_script 
  @language = N'Python', 
  @script = N'import pip
import pandas as pd
installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
   for i in installed_packages])
df = pd.DataFrame(installed_packages_list)
OutputDataSet = df'
WITH RESULT SETS (( InstalledPackageAndVersion nvarchar (150) ))

<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true">1.2 Programming SQL Server Machine Learning Services</h2>

You have two methods of working with Machine Learning Services in SQL Server:

1. You can create your R or Python code on a workstation with the Microsoft R or Python Libraries installed, which run certain operations on the SQL Server Instance remotely.
2. You can "wrap" the R or Python code in a Stored Procedure server-side, and run standard Transact-SQL statements to call for the scoring.

You can use three methods of running the Models you create in Machine Learning Services:

1. Using the Extensibility Framework, you can create trained Machine Learning Models, and store them as a binary object in a SQL Server table. You can then "score" (do the predictions or classifications) in SQL Server by loading the binary model and using it in Python or R code wrapped in a Stored Procedure.
2. Using the Native Scoring feature of the `PREDICT` Transact-SQL statement against a trained Machine Learning Model. The model that you use must have been created using one of the supported algorithms from the RevoScaleR package. 
3. Using the `sp_rxPredict` stored procedure provided as a wrapper for the `rxPredict` R function in RevoScaleR and MicrosoftML, and the `rx_predict` Python function in revoscalepy and microsoftml. It is written in C++ and is optimized specifically for scoring operations.
 

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

- Primary Documentation: [https://docs.microsoft.com/en-us/sql/advanced-analytics/r/sql-server-r-services?view=sql-server-2017](https://docs.microsoft.com/en-us/sql/advanced-analytics/r/sql-server-r-services?view=sql-server-2017)

- https://microsoft.github.io/sql-ml-tutorials/R/customerclustering/

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Project Methodology and Data Science</p>

Next, you'll learn more about setting up your project structure and working with Data Science. Open that Notebook to continue.
